In [ ]:
# Sample queries on Redshift cluster as needed 
# https://docs.aws.amazon.com/redshift/latest/dg/REGEXP_SUBSTR.html
# https://docs.aws.amazon.com/redshift/latest/dg/r_CASE_function.html
# https://docs.aws.amazon.com/redshift/latest/dg/r_REPLACE.html

# Regex
# https://regex101.com/
# https://stackoverflow.com/questions/7317043/regex-not-operator/7317087
# https://docs.aws.amazon.com/redshift/latest/dg/pattern-matching-conditions-posix.html
# https://stackoverflow.com/questions/41022295/redshift-regular-expression-negative-lookahead-does-not-work

In [ ]:
import psycopg2
import configparser

In [ ]:
config = configparser.ConfigParser()
config.read_file(open('./../setup/aws.cfg'))

In [ ]:
conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))

In [ ]:
cursor = conn.cursor()

In [ ]:
# Check parsing errors from json files
cursor.execute("""
    SELECT * 
    FROM stl_load_errors
""")
for record in cursor:
    print(record)

In [ ]:
# Check top 5 from metadata
cursor.execute("""
    SELECT * 
    FROM staging.metadata
    LIMIT 3
""")
for record in cursor:
    print(record)

In [ ]:
cursor.execute("""
    SELECT * 
    FROM staging.classifications
    LIMIT 5
""")
for record in cursor:
    print(record)

In [ ]:
cursor.execute("""
    SELECT * 
    FROM staging.citations
    LIMIT 5
""")
for record in cursor:
    print(record)

In [ ]:
cursor.execute("""
    SELECT * 
    FROM staging.authors
    LIMIT 5
""")
for record in cursor:
    print(record)

In [ ]:
data = []
# regexp_substr("journal-ref",'(?<![0-9-])(19[6-9][0-9])(?![0-9-])|(?<![0-9-])(20[0-9]{2})(?![0-9-])') -- Negative lookbehind/lookahead does not work in Redshift :(      
# https://stackoverflow.com/questions/41022295/redshift-regular-expression-negative-lookahead-does-not-work
# https://docs.aws.amazon.com/redshift/latest/dg/REGEXP_SUBSTR.html
cursor.execute("""
    SELECT 
        "journal-ref",
        regexp_substr("journal-ref",'(19[6-9][0-9])|(20[0-9]{2})')
    from staging.metadata
    order by id
    limit 100
""")
for record in cursor:
    data.append(record[0])
    print(record)

In [ ]:
cursor.execute("""
    SELECT * 
    FROM public.articles_fact
    LIMIT 2
""")
for record in cursor:
    print(record)

In [ ]:
cursor.execute("""
    SELECT * 
    FROM public.articles_fact AS fct
    INNER JOIN public.versions_dim AS ver
        ON ver.article_id = fct.article_id
    ORDER BY fct.article_id
    LIMIT 3 
""")
for record in cursor:
    print(record)

In [ ]:
# Note: https://arxiv.org/help/faq/references
# Citations before 2007 have style: category/YYMMNNN - e.g. hep-th/9910001
# Citations after 2007 have style: YYMM.NNNN - e.g. 1112.4395

cursor.execute("""
    SELECT 
        CASE WHEN 
    FROM staging.citations
    LIMIT 5
""")
for record in cursor:
    print(record)

In [ ]:
cursor.execute("""
    SELECT * 
    FROM public.articles_fact
    WHERE year < 2000
    LIMIT 5
""")
for record in cursor:
    print(record)

In [ ]:
conn.close()